# Criando features

- Esse notebook é a continuação do [1. EDA](1.%20EDA.ipynb)

# Ìndice
1. Resumo e objetivo
- [Features Básicas](#Features-Básicas)
- [Criando Features Novas](#Criando-Features-Novas)
- [Apêndice](#Apêndice)
- [Referências](#Referências)

### Coisas a serem feitas:

- [ ] Colocar todas as referências.
    - [x] [Falta a do Machine Learning Mastery.](#Referência)
    - [x] [Referência do algoritmo](#back2)
- [ ] Colocar one-hot.
- Colocar as outras featues.
    - [x] Lagged.  
    - [ ] Meses.  
    - [ ] Médias. 
- [x] Explicar Lagged.
    - Explicar o descarte dos Nan's.
- [ ] Tem que melhorar muito a explicação das laggeds.

# Carregando os arquivos

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

path = 'data'

train = pd.read_csv(path + '/sales_train.csv.gz')
test = pd.read_csv(path + '/test.csv.gz').set_index('ID')
items = pd.read_csv(path + '/items.csv')
items_cat = pd.read_csv(path + '/item_categories.csv')
shops = pd.read_csv(path + '/shops.csv')

# Features Básicas

#### A feature categórica shop_name, também possui informação sobre localização e tipos dos shoppings.

- Адыгея ТЦ "Мега"         => Adygea shopping center "Mega"	
- Волжский ТЦ "Волга Молл" => Volzhsky shopping center "Volga Mall"
- Вологда ТРЦ "Мармелад"   => Shopping center Vologda "Marmalade"	
- Якутск ТЦ "Центральный"  => Centro comercial Yakutsk "Central"	
- Ярославль ТЦ "Альтаир"   => Centro comercial Yaroslavl "Altair"

In [60]:
# Corrigindo uma entrada.
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'

# Separando as palavras (split) e escolhendo a primeira (map(lambda x: x[0])).
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])

#Corringindo um nome.
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'

print('Nome das cidades:\n',shops['city'].head())

Nome das cidades:
 0      Якутск
1      Якутск
2      Адыгея
3    Балашиха
4    Волжский
Name: city, dtype: object


> ### Obs.:
> Note que é possível retirar mais informações dos nomes, como o tipo de estabelecimento (Shopping, Centro comercial, e etc), o tipo do shopping (Volga Mall) e até mesmo a localização na cidade (Central).

Apesar de termos as localizações separadas, a maioria dos algoritmos só enchergam números, sendo o XGboost uma excessão. Então, temos que rotular cada nome de cidade com um número.

<a name="back1"></a> Deixei um exemplo simples [aqui](#go1) para quem ainda está começando.

In [61]:
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
shops.drop(['shop_name', 'city'], axis='columns', inplace=True)
shops.head()

,shop_id,city_code
0,0,29
1,1,29
2,2,0
3,3,1
4,4,2


O mesmo procedimento vai ser realizado para os nomes das categorias. Vamos dividi-las em dois tipo e subtipo, como por exemplo:
    - Аксессуары - PS2 => Tipo: Acessórios / Subtipo: PS2.
    
Da mesma forma iremos colocar rótulos em cada um dos tipos e subtipos.

In [62]:
# Separando as duas categorias.
items_cat['split'] = items_cat['item_category_name'].str.split('-')

# Criando uma coluna só com o tipo e depois criando os rótulos.
items_cat['type'] = items_cat['split'].map(lambda x: x[0].strip())
items_cat['type_code'] = LabelEncoder().fit_transform(items_cat['type'])

# Criando uma coluna só com o subtipo e depois criando os rótulos.
items_cat['subtype'] = items_cat['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
items_cat['subtype_code'] = LabelEncoder().fit_transform(items_cat['subtype'])

items_cat.head()

,item_category_name,item_category_id,split,type,type_code,subtype,subtype_code
0,PC - Гарнитуры/Наушники,0,"[PC , Гарнитуры/Наушники]",PC,0,Гарнитуры/Наушники,29
1,Аксессуары - PS2,1,"[Аксессуары , PS2]",Аксессуары,1,PS2,9
2,Аксессуары - PS3,2,"[Аксессуары , PS3]",Аксессуары,1,PS3,10
3,Аксессуары - PS4,3,"[Аксессуары , PS4]",Аксессуары,1,PS4,11
4,Аксессуары - PSP,4,"[Аксессуары , PSP]",Аксессуары,1,PSP,13


In [63]:
# Retirando colunas que não tem mais utilidade.
items_cat.drop(['item_category_name', 'split', 'type', 'subtype'], axis='columns', inplace=True)
items_cat.head()

,item_category_id,type_code,subtype_code
0,0,0,29
1,1,1,9
2,2,1,10
3,3,1,11
4,4,1,13


 Vou retirar duas colunas dos bancos de dados, item_name e date. As datas não vão ser precisas porque vou utilizar somente os meses e para isso já temos a coluna date_block_num. O nome dos itens provavelmente tem alguma coisa interessante, mais vai ser muito complicado separar algum informação útil de lá.

In [66]:
items.drop(['item_name'], axis=1, inplace=True)

In [67]:
train.drop(['date'], axis='columns', inplace=True)

# Criando Features Novas

Criar features é uma das coisas mais importantes a se fazer. Uma boa feature ajuda o modelo a fazer melhores previsões. Vou abordar a construção das seguintes features:

- [Revenue](#Revenue)
- [Meses](#Meses)  
- [Médias](#Médias)
- [Lagged Variables](#Lagged-Variables) 

### Revenue

#### A feature mais básica de todas é a receita (Dinheiro total ganho por item vendido).

In [73]:
train['revenue'] = train['item_price'] * train['item_cnt_day']

--------------------------------

### Meses

A ideia aqui é criar colunas que indicam meses especiais com 1 e meses normais com 0. Por exemplo, podemos criar uma coluna onde, em todos os anos, o mês de Dezembro é indicado com 1. Isso deve ajudar o algoritmo a prever o feriado de natal.

In [173]:
# Criando uma coluna com zeros.
train['Dezembro'] = 0

# Nos lugares onde o date_block_num for divisível por 12 colocamos o valor 1.
train['Dezembro'][train.date_block_num % 12 == 0]  = 1

/home/ronaldo/anaconda3/envs/Data_Science/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


> <a name="back3"></a> **Obs.:** Se ficou muito complicado de entender o que aconteceu na última linha, eu coloquei um exemplo mais simples [aqui](#mask).

In [179]:
# Checando se os valores foram substituidos corretamente.
train[train.date_block_num == 12].head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue,Dezembro
1267562,12,54,10297,749.0,1.0,749.0,1
1267563,12,54,10297,749.0,1.0,749.0,1
1267564,12,54,10297,749.0,1.0,749.0,1
1267565,12,54,10296,1599.0,1.0,1599.0,1
1267566,12,54,10297,749.0,1.0,749.0,1


In [ ]:
# Criando uma coluna com zeros.
train['Maio'] = 0

# Nos lugares onde o date_block_num for divisível por 12 colocamos o valor 1.
train['Maio'][train.date_block_num + 1 % 6 == 0]  = 1

---------------------------

### Lagged Variables

Uma lagged feature é obtida ao atrasarmos ou adiantarmos a sequência daquela feature. Como por exemplo,

| Vendas_original  | Vendas_lag1    |
|----------|:------------- |
| 1 |  Nan |
| 2 |    1   |
| 3 | 2 |

A ideia por trás de fazermos essa transformação, é dar ao algoritmo a oportunidade de fazer escolhas como:
- O número de vendas desse mês é o mesmo número do mês passado.
- Dependendo do tamanho do lag é possível que o algoritmo aprenda sobre sazonalidade das vendas. Um bom exemplo é o natal, todo ano existe um aumento nas vendas neste período e o algoritmo pode levar isso em consideração.

In [101]:
# Criando um pandas dataframe com o número de vendas de 1 até 6.
df_aux = pd.DataFrame(np.linspace(1,6,6), columns=['Vendas'])
df_aux

,Vendas
0,1.0
1,2.0
2,3.0
3,4.0
4,5.0
5,6.0


> **Obs.:** df_aux simula um dataset das vendas de uma loja, onde a passagem de tempo seria algo do tipo:
- linha 0 = mês de Janeiro.
- linha 1 = mês de Fevereiro.

In [102]:
#Usando a função shift para deslocar o array em 1.
df_aux['Vendas_shift_+'] = df_aux['Vendas'].shift(1)
df_aux['Vendas_shift_-'] = df_aux['Vendas'].shift(-1)

df_aux

,Vendas,Vendas_shift_+,Vendas_shift_-
0,1.0,NaN,2.0
1,2.0,1.0,3.0
2,3.0,2.0,4.0
3,4.0,3.0,5.0
4,5.0,4.0,6.0
5,6.0,5.0,NaN


Temos que descartar as linhas que possuem Nan's.

In [103]:
df_aux.dropna(inplace=True)
df_aux

,Vendas,Vendas_shift_+,Vendas_shift_-
1,2.0,1.0,3.0
2,3.0,2.0,4.0
3,4.0,3.0,5.0
4,5.0,4.0,6.0


Note que as colunas estão fora de ordem, para que o algoritmo reconheça a passagem de tempo é necessário reorganziar as colunas.

In [104]:
df_aux = df_aux[['Vendas_shift_+', 'Vendas', 'Vendas_shift_-']]
df_aux

,Vendas_shift_+,Vendas,Vendas_shift_-
1,1.0,2.0,3.0
2,2.0,3.0,4.0
3,3.0,4.0,5.0
4,4.0,5.0,6.0


É mais conveninente renomear as colunas tmb.

In [105]:
df_aux.rename(columns={'Vendas_shift_+': 'Vendas_t-2',
                       'Vendas': 'Vendas_t-1',
                       'Vendas_shift_-': 'Vendas_t'}
              , inplace=True)
df_aux

,Vendas_t-2,Vendas_t-1,Vendas_t
1,1.0,2.0,3.0
2,2.0,3.0,4.0
3,3.0,4.0,5.0
4,4.0,5.0,6.0


Compare essa tabela com a gerada inicialmente, onde a passagem dos meses seguiam o número das linhas. Na nova tabela, a passagem de tempo pode ser vista seguindo as colunas $t-2$, $t-1$, $t$.

> **Obs.:** Esse tipo de organização é perfeita para usarmos em algoritmo de Machine Learning. As linhas seriam suas amostras, as features seriam $t-2$ e $t-1$, e o que você estaria prevendo seria $t$.

<a name="back2"></a>  Vamos voltar ao dataset que queremos utilizar. O processo de criar lagged features é bem chato se for feito manualmente igual fizemos acima, principalmente se fizermos com muitas variáveis. Acabei fazendo um função pra fazer isso ([time_shift](#lagged)). Ela foi baseado em outra função encontrada [aqui][lag] (ótimo site para aprender Machine Learning).

[lag]: https://machinelearningmastery.com/basic-feature-engineering-time-series-data-python/

,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue
0,0,59,22154,999.00,1.0,999.00
1,0,25,2552,899.00,1.0,899.00
2,0,25,2552,899.00,-1.0,-899.00
3,0,25,2554,1709.05,1.0,1709.05
4,0,25,2555,1099.00,1.0,1099.00


-----------------

# Apêndice

------------------------------

<a name="go1"></a>No código abaixo, x vai guardar a relações entre os rótulos,
-  Ex.: a = 1, b = 2, e etc.

Isso é importante se quisermos fazer a transformação reversa.

[Voltar ao notebook](#back1)

In [49]:
x = LabelEncoder()

vetor_normal = ['a','b','c','d']

vetor_transformado = x.fit_transform(vetor_normal)

print('Categorias', vetor_normal)
print('Rótulos:', vetor_transformado)

voltando = x.inverse_transform(vetor_transformado)

print('Invertendo a transformação:', voltando)

Categorias ['a', 'b', 'c', 'd']
Rótulos: [0 1 2 3]
Invertendo a transformação: ['a' 'b' 'c' 'd']


----------------------------------------

### <a name="lagged"></a> Código para gerar e organizar as variáveis com lag.

[Voltar ao notebook](#back2)

In [72]:
def time_shift(df, lags, dropnan=True):
    
    '''
    Adiciona novas colunas deslocadas no tempo.
    Argumentos:
    
        df: Pandas DataFrame.
        
        lags: lista com o número de passos
               a ser deslocado.
               
        dropnan: (True) Retira as linhas que
                teham Nan's.
                
    Retorna:
        Pandas DataFrame com as colunas deslocadas.
    '''
    lags.append(0)
    lags = -1 * np.sort(lags)
    
    columns = [str(df.name)] if type(df) is pd.Series else df.columns
    col = []
    
    for column in columns:
        for lag in lags: 
            
            if lag == 0:
                new_column = column 
                col.append(new_column)
            else:
                new_column = column + '_lag(' + str(-1 * lag) + ')'
                col.append(new_column)
                df[new_column] = df[column].shift(lag)
                
    if dropnan:
        df.dropna(inplace=True)
        
    return df[col]

### <a name="mask"></a> Exemplo de seleção para o pandas.

[Voltar ao notebook](#back3)

Vamos criar um array com números de 1 a 36.

In [182]:
month = np.linspace(1,36,36)
month

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30., 31., 32., 33., 34., 35., 36.])

Ao fazermos (month % 12), o python retorna o resto da divisão do array month por 12, ou seja, os únicos números que divisíveis são 12, 24, 36.

In [186]:
(month % 12)

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.,  0.,  1.,
        2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.,  0.,  1.,  2.,
        3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.,  0.])

Agora, ao adicionarmos uma comparação (month % 12 == 0), queremos saber em quais posições do arrya a condição (== 0) é verdade.

In [185]:
(month % 12 == 0)

array([False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False,  True])

Logo, ao colocarmos esse tipo de array no pandas queremos dizer: Só faça o que pedimos nas posições onde temos **True**, não faça nada se for **False**.

----------------------------------------

# Referências
1. **Muitas das features craidas neste notebook foram retiradas deste [kernel](https://www.kaggle.com/dlarionov/feature-engineering-xgboost).**
2. **Site de onde tirei as informações sobre lagged variables. [Machine Learning Mastery](https://machinelearningmastery.com/basic-feature-engineering-time-series-data-python/)**